In [1]:
import pandas as pd
import statistics

In [2]:
# Datos demográficos
demog_df=pd.read_csv('1c_COVID19_demographics_cleaned_deid.csv')

print ('Demographic: ')
print(demog_df.shape)

Demographic: 
(1518, 50)


In [3]:
# Datos sueño
psqi_df=pd.read_csv('2b_COVID19_Round1_cleaned_deid_2020-12-10_15_24.csv')

print ('PSQI data')
print (psqi_df.shape)

PSQI data
(839, 263)


In [4]:
# Merge data
merged_df=psqi_df.merge(demog_df, how='inner', on='sub_id')

# Quitar missing values (cuando todos los valores son NaN)
merged_df.dropna(axis='columns',how='all',inplace=True)

print ('Merged data')
print (merged_df.shape)

Merged data
(839, 311)


In [5]:
# Remplazar missing values por valores específicos - Demographic variables

merged_df.preferred_gender = merged_df.preferred_gender.fillna(5)
merged_df.transgender2 = merged_df.transgender2.fillna(3)
merged_df.sexual_orientation = merged_df.sexual_orientation.fillna(5)
merged_df.dependent_children=merged_df.dependent_children.fillna(0)

merged_df['psqi_2_mean']= merged_df.groupby('age1')['psqi_2'].transform('mean')
merged_df['psqi_4_mean']=merged_df.groupby('age1')['psqi_4'].transform('mean')
merged_df['psqi_2'].fillna(merged_df['psqi_2_mean'], inplace=True)
merged_df['psqi_4'].fillna(merged_df['psqi_4_mean'], inplace=True)


In [6]:
#Drop variables
vars_to_drop=list(merged_df.filter(regex=r'(PSQI|CERQ|LSAS|Big|mtq)')) 
unwanted=['psqi_2_mean','psqi_4_mean','race_description','year_study','state','gad_7_total',
              'round_1_timestamp','country','time_zone','gender_description','so_description','psqi_2NEW',
              'round_1_complete','record_id_x','record_id_y','date_time','date_time_rd1']
for ele in unwanted:
    vars_to_drop.append(ele)
    
wanted=['PSQI_TOTAL','mtq_precovid_workdays','mtq_precovid_workday_sleeponset',
      'mtq_precovid_workday_sleepend','mtq_precovid_freeday_sleeponset',
      'mtq_precovid_freeday_sleepend','mtq_postcovid_workdays',
      'mtq_postcovid_workday_sleeponset','mtq_postcovid_workday_sleepend',
      'mtq_postcovid_freeday_sleeponset','mtq_postcovid_freeday_sleepend']
for ele in wanted:
    vars_to_drop.remove(ele)

for ele in vars_to_drop:
    merged_df.drop(columns=ele,inplace=True)

In [7]:
# Remplazar missing values (NaN) con la moda - Variables del sueño
vars_to_keep=list(merged_df.filter(regex=r'(psqi|gad|GAD|cerq|fear|avoid|big|employed|working_home|employment_covid)')) 
unwanted=['psqi_1','psqi_2','psqi_3','psqi_4']
mtq_wanted=wanted
wanted=['PSQI_TOTAL','mtq_precovid_workdays','mtq_postcovid_workdays']

for ele in unwanted:
    vars_to_keep.remove(ele)
    if ele=='psqi_1' or ele=='psqi_3':
        mtq_wanted.append(ele)

for ele in wanted:
    vars_to_keep.append(ele)
    mtq_wanted.remove(ele)

for var_name in vars_to_keep:
    #calculate_mode(var_name)
    merged_df[var_name+'_mode']=merged_df[var_name].mode()
    merged_df[var_name].fillna(merged_df[var_name+'_mode'][0], inplace=True)
    merged_df.drop(columns=var_name+'_mode',inplace=True)

# Cambiar formato de horas a minutos en variables temporales
for ele in mtq_wanted:
    merged_df[ele] = merged_df[ele].fillna(statistics.mode(merged_df[ele]))
    merged_df[ele] = merged_df[ele].apply(lambda x: (int(str(x)[0:2])*60) + int(str(x)[3:5]))

# Crear nuevas variables
merged_df['mtq_dif_workday_sleeponset'] = merged_df['mtq_precovid_workday_sleeponset'] - merged_df['mtq_postcovid_workday_sleeponset']
merged_df['mtq_dif_workday_sleepend'] = merged_df['mtq_precovid_workday_sleepend'] - merged_df['mtq_postcovid_workday_sleepend']
merged_df['mtq_dif_freeday_sleeponset'] = merged_df['mtq_precovid_freeday_sleeponset'] - merged_df['mtq_postcovid_freeday_sleeponset']
merged_df['mtq_dif_freeday_sleepend'] = merged_df['mtq_precovid_freeday_sleepend'] - merged_df['mtq_postcovid_freeday_sleepend']


In [8]:
# Evaluar si ya no hay missing values
columns = merged_df.columns
n_values = [merged_df[a].unique() for a in merged_df.columns]
cuenta = pd.DataFrame()
cuenta['features'] = columns
cuenta['n_values'] = n_values

missing_values=merged_df.columns[merged_df.isnull().any()]
print (merged_df.shape)

(839, 225)


In [9]:
#One hot encoding (aunque en realidad ya no hay variables categóricas, solo para estar seguras)
merged_df=pd.get_dummies(merged_df, drop_first=True) 

In [10]:
# Guardar dataset limpio en formato csv
merged_df.to_csv('data_no_missing_values.csv', index=False)